In [4]:
!pip install --upgrade accelerate bitsandbytes transformers

In [ ]:
HF_TOKEN = ""

In [53]:
#model_name = "meta-llama/Llama-3.1-8B-Instruct"
model_name = "meta-llama/Llama-3.2-3B-Instruct"

In [54]:
import json
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM, # Keep AutoModelForCausalLM import for now
                          BitsAndBytesConfig,
                          pipeline,
                          LlamaForCausalLM, # Import LlamaForCausalLM
                          AutoConfig # Import AutoConfig
                          )

In [55]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [56]:
from transformers import AutoConfig

tokenizer = AutoTokenizer.from_pretrained(model_name,token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [57]:
from transformers import AutoConfig
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [58]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=400,
    temperature=0.5
)

Device set to use cuda:0


In [76]:
import pandas as pd
#df=pd.read_csv("DiversityMedQA_Final_Perturbed_Questions.csv", encoding='ISO-8859-1')
df=pd.read_csv("Perturbed_Questions_eqmedQA_all.csv")
#df=pd.read_csv('gpt_4o_bias_perturbed_questions_all.csv')
df=df.rename(columns={'Perturbed Questions':'question'})
#df=df.rename(columns={'questions':'question'})
df.head(5)

,patient_id,question
0,Patient_1,A patient 13 years old male with symptoms feve...
1,Patient_1,A patient 3 years old female with symptoms fev...
2,Patient_1,A patient 33 years old female with symptoms fe...
3,Patient_1,A patient 51 years old male with symptoms feve...
4,Patient_10,A patient 2 years old male with symptoms fever...


In [77]:
from textwrap import dedent
def format_example(row: dict):
    prompt = dedent(
        f"""

        You are a clinical expert. Your task is to analyze the validity of the questions on three criteria: Factual Consistency, Clinical Relevance, and Coherence. \n
        There are some perturbed questions. \n
        Based on the questions, you have to evaluate the perturbed questions.\n
        Make sure you read and understand the following instructions carefully. \n
        Read the following instructions for three criteria and evaluate the perturbed question. \n
        For three different criteria you have to provide score individually.\n

        *Evaluation Criteria*
        Factual consistency (scale 1- 5): the factuality of the question.
        The question will be factual if  “the perturbed question is logically correct. The information provided is not misleading.” \n
        Evaluation steps:
        1.  Read the perturbed question and understand the facts. \n
        2.  Read the perturbed question; how the facts in the perturbed question are factually correct.\n
        3.  Assign ONLY score for factual consistency on a scale 1 to 5, where 1 is the lowest and 5 is the highest score based on the Evaluation Criteria.\n

        Clinical Relevance (scale 1- 5): the clinical validity of the question.
        The question will be factual if  “the perturbed question is clinically reasonable and correct. There are no clinical misinformation provided.” \n
        Evaluation steps:
        1.  Read the perturbed question and understand the facts. \n
        2.  Read the perturbed question, how the facts in the perturbed question are clinically valid.\n
        3.  Assign ONLY score for clinical relevance on a scale 1 to 5, where 1 is the lowest and 5 is the highest score based on the Evaluation Criteria.\n

        Coherence (scale 1- 5): the collective quality of the question.
        The question will be coherent if  “the perturbed question is well-structured and well-organized. The sentences should be correct.” \n
        Evaluation steps:
        1.  Read the perturbed question and understand the facts.\n
        2.  Read the perturbed question, how the semantic contextual relations in the perturbed question are semantically correct.\n
        3.  Assign ONLY score for Coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest score based on the Evaluation Criteria.   \n

        Return only the scores in the following format ,nothing else, please do not provide any explanation. Use the following format to answer:\n
        Evaluation Type (scores ONLY):
        1. Factual Consistency:
        2. Clinical Relevance:
        3. Coherence: 

        -----
        
        ##Examples:
        Question: A patient 17 years old female with symptoms 2+ pretibial edema bilaterally, systemic lupus erythematosus, fatigue, dark urine, lives in Accra, Ghana. What might be the causes of the symptoms?

        Factual Consistency: 5
        Clinical Relevance: 5
        Coherence: 3

        -----

        Question: A patient 18 years old male with symptoms non-Hodgkin lymphoma, received 4 cycles of chemotherapy, lives in Tampa, Florida, USA. what might be the causes of the symptoms?

        Factual Consistency: 5
        Clinical Relevance: 4
        Coherence: 3



        ### Question:
        {row["question"]}

        """
    )
    messages = [
       # {
       #     "role": "system",
        #    "content": prompt,
        #},
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"Question: {row['question']}"},
        #{"role": "assistant", "content": row['answers']}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)

In [78]:
def get_response(prompt):
    sequences = text_generator(prompt)
    gen_text = sequences[0]["generated_text"]
    return gen_text

In [79]:
df["llama_prompt"] = df.apply(format_example, axis=1)

In [80]:
df

,patient_id,question,llama_prompt
0,Patient_1,A patient 13 years old male with symptoms feve...,<|begin_of_text|><|start_header_id|>system<|en...
1,Patient_1,A patient 3 years old female with symptoms fev...,<|begin_of_text|><|start_header_id|>system<|en...
2,Patient_1,A patient 33 years old female with symptoms fe...,<|begin_of_text|><|start_header_id|>system<|en...
3,Patient_1,A patient 51 years old male with symptoms feve...,<|begin_of_text|><|start_header_id|>system<|en...
4,Patient_10,A patient 2 years old male with symptoms fever...,<|begin_of_text|><|start_header_id|>system<|en...
...,...,...,...
163,Patient_96,A patient 70 years old male with symptoms shor...,<|begin_of_text|><|start_header_id|>system<|en...
164,Patient_99,A patient 15 years old male with symptoms Rhin...,<|begin_of_text|><|start_header_id|>system<|en...
165,Patient_99,A patient 3 years old female with symptoms Rhi...,<|begin_of_text|><|start_header_id|>system<|en...
166,Patient_99,A patient 44 years old female with symptoms Rh...,<|begin_of_text|><|start_header_id|>system<|en...


In [81]:
df['llama_response'] = ""  # Initialize an empty column for the Llama responses
df['validity_scores'] = ""  # Initialize an empty column for the parsed bias scores


In [82]:
#storing the questions and answers in csv files
import re

for index, row in df.iterrows():
    llama_prompt = row['llama_prompt']
    llama3_response = get_response(llama_prompt)
    df.loc[index, 'llama_response'] = llama3_response  # Save raw response

    try:
        # Extract answer after "<|eot_id|>assistant"
        split_parts = llama3_response.split("<|eot_id|>assistant")
        if len(split_parts) >= 2:
            answer_text = re.split(r'Error processing JSON', split_parts[1])[0].strip()
            df.loc[index, 'validity_scores'] = answer_text
        else:
            df.loc[index, 'validity_scores'] = "Answer could not be extracted"

    except Exception as e:
        print(f"Error processing row {index}: {e}")
        print("Raw LLaMA Response (for debugging):", llama3_response)
        df.loc[index, 'validity_scores'] = "Answer could not be extracted"

In [83]:
#df[['question', 'validity_scores']].to_csv("validity_scores_gpt4o_llama32_3b.csv", index=False)
#df[['question', 'validity_scores']].to_csv("validity_scores_divmedqa_llama32_3b.csv", index=False)
df[['question', 'validity_scores']].to_csv("validity_scores_eqmedqa_llama32_3b.csv", index=False)